## GA3C-CADRL
#### GPU/CPU Asynchronous Advantage Actor-Critic for Collision Avoidance with Deep Reinforcement Learning
Michael Everett, Yu Fan Chen, and Jonathan P. How<br>
Manuscript submitted to 2018 IEEE/RSJ International Conference on Intelligent Robots and Systems (IROS)

**Objective:** This goal of this notebook is to explain how to use our code, enabling other researchers to test and compare against the results presented in the paper. After reading this notebook, it should also be clear how our code could be implemented on your own system (i.e. what format you should provide as input, and what information you'll get as output)

## Create an instance of an Agent
The most important class is Agent, which has attributes such as radius and position, and methods such as find_next_action. The environment is made up of several Agents.

In [8]:
import agent
import network
import util
import numpy as np
np.random.seed(0)

### Load trained network

In [2]:
possible_actions = network.Actions()
num_actions = possible_actions.num_actions
nn = network.NetworkVP_rnn(network.Config.DEVICE, 'network', num_actions)
nn.simple_load('../checkpoints/network_01900000')

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ../checkpoints/network_01900000


In [3]:
STEP = 30

In [4]:
paths = np.load('../src/all_not_ext_paths_with_ids.npy')
ped_time = (paths[:, 1] / 20).astype(int)
ped_col = (paths[:, 2] / STEP).astype(int)
ped_row = (paths[:, 3] / STEP).astype(int)

In [6]:
grid = np.load('../grid_with_static_obstacles.npy')

In [10]:
gen_paths = np.load('../src/gen_paths.npy')

In [64]:
max_row, max_col = np.array(grid.shape)

In [65]:
boundaries = np.array(
    [(i, j) for i in range(-2, 0) for j in range(-2, max_col + 2)] + \
    [(i, j) for i in range(max_row, max_row + 2) for j in range(-2, max_col + 2)] + \
    [(i, j) for i in range(-2, max_row + 2) for j in range(-2, 0)] + \
    [(i, j) for i in range(-2, max_row + 2) for j in range(max_col, max_col + 2)]
)

In [151]:
class Robot:
    def __init__(self, grid=grid, ped_time=ped_time, ped_col=ped_col, ped_row=ped_row, boundaries=boundaries):
        self.grid = grid
        self.ped_time = ped_time
        self.ped_col = ped_col
        self.ped_row = ped_row
        self.collisions = 0
        self.boundaries = boundaries
        self.trace = []
            
    def neighbor_cells(self, i, j, r):
        i, j = int(i), int(j)
        res = [(x, j-r) for x in range(i-r, i+r+1)] + [(x, j+r) for x in range(i-r, i+r+1)]
        res += [(i-r, y) for y in range(j-r+1, j+r)] + [(i+r, y) for y in range(j-r+1, j+r)]
        return res
    
    def check_cell(self, time, row, col):
        t = self.ped_time == time
        ped_row_time = self.ped_row[t]
        ped_col_time = self.ped_col[t]
        ind = (ped_row_time == row) * (ped_col_time == col)
        return ind.any()
    
    def get_obstacles(self, time, row, col):
        obstacles = []
        neighbors = self.neighbor_cells(row, col, 1)
        for x, y in neighbors:
            if x < grid.shape[0] and y < grid.shape[1]:
                if self.grid[x, y] or self.check_cell(time, x, y):
                    obstacles.append(agent.Agent(x, y, x+5, y+5, 0.49, 0, 0, i+1))
        for x, y in self.boundaries:
            obstacles.append(agent.Agent(x, y, x+5, y+5, 0.49, 0, 0, i+1))
            
        return obstacles
        
    def get_path(self, start_time, row_start, col_start, row_goal, col_goal):
        self.collisions = 0
        self.obstacles = []
        
        time = start_time
        
        self.trace = []

        row_cur, col_cur = row_start, col_start

        other_agents = []

        host_agent = agent.Agent(row_cur, col_cur, row_goal, col_goal, 0.49, 1.4, 0, 0)
        host_agent.vel_global_frame = np.array([0, 0])

        while np.abs(row_cur - row_goal) > 0.4 or np.abs(col_cur - col_goal) > 0.4:

            if self.check_cell(time, np.round(row_cur), np.round(col_cur)):
                self.collisions += 1
            
            other_agents = self.get_obstacles(time, row_cur, col_cur)

            obs = host_agent.observe(other_agents)[1:]
            obs = np.expand_dims(obs, axis=0)

            predictions = nn.predict_p(obs, None)[0]
            raw_action = possible_actions.actions[np.argmax(predictions)]

            host_agent.update_state(raw_action, 1)
            host_agent.pos_global_frame = np.round(host_agent.pos_global_frame)
            row_cur, col_cur = host_agent.pos_global_frame
            self.trace.append([row_cur, col_cur])

            time += 1
            
            if time - start_time > 200 \
                    or row_cur >= self.grid.shape[0] or row_cur < 0 \
                    or col_cur >= self.grid.shape[1] or col_cur < 0:
                return None
                
        return self.trace

In [152]:
robot = Robot()

In [171]:
paths = []
delays = []
collisions = []

for i, (start_time, row_start, col_start, row_goal, col_goal, opt_time) in enumerate(gen_paths):

    if i in fails:
        paths.append(None)
        collisions.append(0)
        continue
    
    result = robot.get_path(start_time, row_start, col_start, row_goal, col_goal)
    
    if result is None:
        paths.append(None)
        collisions.append(0)
        print(i, " fail")
        continue
    
    paths.append(result)
    collisions.append(robot.collisions)
    if opt_time == 0:
        delays.append(1)
        print(i)
    else:
        delays.append(1. * len(result) / opt_time)
    
    if i % 10 == 0:
        print(i)

0
10
20
40
50
70
80
100
120
130
140
150
160
170
180
190
200
210
220
230
240
270
280
290
310
320
330
339
340
350
360
370
380
400
410
420
430
440
450
460
490
500
520
530
550
560
560
580
590
600
610
620
640
650
660
670
690
700
710
720
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970


In [172]:
collisions = np.array(collisions)
print(collisions.sum())
np.save('net019_round_speed1.4/collisions', collisions)

410


In [176]:
delays = np.array(delays)
print(delays.mean())
np.save('net019_round_speed1.4/delays', delays)

2.329641857605303


In [190]:
delays[delays < 7].mean()

1.575057875621655